In [1]:
import os

In [2]:
%pwd

'd:\\Piyush\\End-to-end-classification-project-pipeline-\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Piyush\\End-to-end-classification-project-pipeline-'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config




In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

[2024-05-06 07:54:14,803: INFO: utils: NumExpr defaulting to 8 threads.]


In [10]:
# This code defines a class PrepareCallback that encapsulates the creation of callbacks for TensorBoard logging and model checkpointing
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config


    '''This property generates a TensorBoard callback. 
    It creates a unique log directory for TensorBoard logs based on the current timestamp and 
    the root log directory specified in the config. It returns a TensorBoard callback instance configured with the log directory.'''
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    '''This property creates a ModelCheckpoint callback. It returns a ModelCheckpoint callback instance configured with the
      checkpoint file path specified in the config. 
    The save_best_only parameter is set to True, indicating that only the best model (based on validation performance) will be saved.'''
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )

    '''This method returns a list containing both TensorBoard and ModelCheckpoint callbacks. It essentially combines both types of callbacks into a single list for ease of use.'''
    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]


In [11]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
    
except Exception as e:
    raise e

[2023-05-31 14:37:12,294: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-05-31 14:37:12,303: INFO: common: yaml file: params.yaml loaded successfully]
[2023-05-31 14:37:12,305: INFO: common: created directory at: artifacts]
[2023-05-31 14:37:12,307: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2023-05-31 14:37:12,310: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]


In [12]:
import time
timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
f"tb_logs_at_{timestamp}"


'tb_logs_at_2023-05-31-14-38-20'